In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../..")

import pandas as pd
from loguru import logger

import src.preprocessing.functions as preprocessing_functions
from src.global_vars import BASE_DATA_DIR

data_root_dir = f"{BASE_DATA_DIR}/sun_et_al_data/"
columns_to_keep = ["Sample", "Group", "Project", "Project_1"]
studies_to_remove = ["LiS_2021a", "LiS_2021b"]
top_250_features = ['s__Anaerobutyricum_hallii', 's__Fusicatenibacter_saccharivorans', 's__Ruminococcus_torques', 's__Bilophila_wadsworthia', 's__Anaerostipes_hadrus', 's__Dorea_longicatena', 's__Phocaeicola_vulgatus', 's__Blautia_wexlerae', 's__Agathobaculum_butyriciproducens', 's__Phocaeicola_dorei', 's__Prevotella_copri_clade_A', 's__Lacrimispora_amygdalina', 's__Dysosmobacter_welbionis', 's__Faecalibacillus_intestinalis', 's__Eubacterium_rectale', 's__Lachnospira_eligens', 's__Collinsella_aerofaciens', 's__Coprococcus_comes', 's__Ruthenibacterium_lactatiformans', 's__Roseburia_faecis', 's__Clostridium_fessum', 's__Bacteroides_ovatus', 's__Phocaeicola_plebeius', 's__Dorea_formicigenerans', 's__Clostridium_sp_AF34_10BH', 's__Roseburia_intestinalis', 's__Blautia_faecis', 's__Clostridia_bacterium', 's__Flavonifractor_plautii', 's__Bacteroides_stercoris', 's__Parabacteroides_distasonis', 's__Escherichia_coli', 's__Faecalibacterium_prausnitzii', 's__Lachnospiraceae_bacterium', 's__GGB9614_SGB15049', 's__Clostridiales_bacterium_KLE1615', 's__Streptococcus_salivarius', 's__Bacteroides_thetaiotaomicron', 's__Bacteroides_caccae', 's__GGB3746_SGB5089', 's__Blautia_massiliensis', 's__Clostridium_sp_AM33_3', 's__Eggerthella_lenta', 's__Gemmiger_formicilis', 's__Phocaeicola_massiliensis', 's__Ruminococcus_gnavus', 's__Firmicutes_bacterium_AF16_15', 's__Clostridia_unclassified_SGB4121', 's__Faecalibacterium_SGB15346', 's__Phascolarctobacterium_faecium', 's__Megamonas_funiformis', 's__Clostridiaceae_bacterium', 's__Bacteroides_xylanisolvens', 's__Roseburia_inulinivorans', 's__Bacteroides_fragilis', 's__Blautia_obeum', 's__Bifidobacterium_longum', 's__Bacteroides_uniformis', 's__Bifidobacterium_pseudocatenulatum', 's__Romboutsia_timonensis', 's__Candidatus_Cibiobacter_qucibialis', 's__Haemophilus_parainfluenzae', 's__Alistipes_onderdonkii', 's__Eubacterium_ramulus', 's__GGB9699_SGB15216', 's__Ruminococcus_bromii', 's__Odoribacter_splanchnicus', 's__Enterocloster_bolteae', 's__Lachnospira_pectinoschiza', 's__Clostridium_sp_AM22_11AC', 's__Parabacteroides_merdae', 's__Clostridium_symbiosum', 's__Anaerotignum_faecicola', 's__GGB9632_SGB15089', 's__Sutterella_wadsworthensis', 's__Hungatella_hathewayi', 's__GGB9581_SGB14999', 's__Eubacterium_ventriosum', 's__Phocaeicola_coprocola', 's__Roseburia_hominis', 's__Parasutterella_SGB9260', 's__Streptococcus_parasanguinis', 's__Bacteroides_cellulosilyticus', 's__Oscillibacter_sp_ER4', 's__GGB9730_SGB15291', 's__Alistipes_putredinis', 's__Alistipes_shahii', 's__Clostridium_sp_AF20_17LB', 's__Butyricicoccus_sp_AM29_23AC', 's__GGB9615_SGB15053', 's__Ruminococcus_bicirculans', 's__Evtepia_gabavorous', 's__Clostridiaceae_bacterium_OM08_6BH', 's__Bacteroides_nordii', 's__Alistipes_indistinctus', 's__Coprococcus_catus', 's__Bacteroides_finegoldii', 's__Faecalicatena_fissicatena', 's__Parasutterella_excrementihominis', 's__Lachnospiraceae_bacterium_WCA3_601_WT_6H', 's__Enterocloster_citroniae', 's__Clostridiales_bacterium', 's__Ruminococcaceae_bacterium', 's__Veillonella_parvula', 's__Lachnospira_sp_NSJ_43', 's__Clostridium_sp_AF27_2AA', 's__Clostridiales_unclassified_SGB15145', 's__Barnesiella_intestinihominis', 's__Bacteroides_intestinalis', 's__Butyricimonas_virosa', 's__Akkermansia_muciniphila', 's__Eubacteriaceae_bacterium', 's__Ruminococcus_lactaris', 's__Klebsiella_pneumoniae', 's__Clostridium_sp_AT4', 's__Coprococcus_eutactus', 's__Clostridium_phoceensis', 's__Alistipes_communis', 's__Clostridia_unclassified_SGB4447', 's__Blautia_stercoris', 's__Blautia_sp_AF19_10LB', 's__Allisonella_histaminiformans', 's__Clostridium_sp_AF15_49', 's__Mesosutterella_multiformis', 's__Enterocloster_aldensis', 's__Bacteroides_salyersiae', 's__GGB9758_SGB15368', 's__Alistipes_finegoldii', 's__Adlercreutzia_equolifaciens', 's__Bifidobacterium_adolescentis', 's__Eubacterium_sp_AF34_35BH', 's__Ruminococcus_callidus', 's__GGB3523_SGB4703', 's__Alistipes_ihumii', 's__Ruminococcaceae_unclassified_SGB15265', 's__Candidatus_Avimicrobium_caecorum', 's__Dysosmobacter_sp_NSJ_60', 's__Lacrimispora_celerecrescens', 's__Tyzzerella_nexilis', 's__Ruminococcaceae_unclassified_SGB15260', 's__Paraprevotella_clara', 's__Blautia_SGB4815', 's__Dialister_hominis', 's__Bacteroides_eggerthii', 's__Alistipes_sp_AF17_16', 's__GGB3256_SGB4303', 's__Ruminococcaceae_unclassified_SGB4425', 's__Erysipelatoclostridium_ramosum', 's__GGB9707_SGB15229', 's__GGB3293_SGB4348', 's__Intestinibacter_bartlettii', 's__Clostridium_sp_AF36_4', 's__Parabacteroides_goldsteinii', 's__Clostridium_leptum', 's__Clostridium_innocuum', 's__GGB9633_SGB15091', 's__Clostridium_sp_AM42_4', 's__Lachnospira_SGB5077', 's__Clostridium_sp_AM49_4BH', 's__GGB9615_SGB15052', 's__Coprobacter_fastidiosus', 's__Veillonella_atypica', 's__Eubacterium_siraeum', 's__Bacteroidaceae_bacterium', 's__Ruminococcus_sp_NSJ_71', 's__Clostridium_sp_AF34_13', 's__Ruminococcaceae_unclassified_SGB15234', 's__Clostridia_unclassified_SGB6276', 's__GGB9627_SGB15081', 's__GGB9705_SGB15225', 's__Clostridia_unclassified_SGB4367', 's__Gemmiger_SGB15295', 's__Roseburia_sp_AF02_12', 's__Holdemania_filiformis', 's__Mediterraneibacter_glycyrrhizinilyticus', 's__GGB9602_SGB15031', 's__Lachnospiraceae_bacterium_BX3', 's__Bacteroides_clarus', 's__GGB9342_SGB14306', 's__Desulfovibrio_piger', 's__Faecalimonas_umbilicata', 's__Blautia_glucerasea', 's__GGB9615_SGB15051', 's__GGB3277_SGB4327', 's__Veillonella_dispar', 's__Ruminococcus_SGB4421', 's__Butyricimonas_faecihominis', 's__GGB3571_SGB4778', 's__Bifidobacterium_bifidum', 's__Dorea_sp_AF36_15AT', 's__Clostridium_SGB6179', 's__GGB9712_SGB15244', 's__Lachnospira_SGB5076', 's__Dialister_invisus', 's__Bacilli_unclassified_SGB6473', 's__Lachnospiraceae_unclassified_SGB4894', 's__Fusobacterium_mortiferum', 's__Megasphaera_sp_BL7', 's__Alistipes_senegalensis', 's__Dorea_sp_AF24_7LB', 's__Intestinimonas_butyriciproducens', 's__Clostridium_scindens', 's__Ligilactobacillus_salivarius', 's__GGB9635_SGB15106', 's__Candidatus_Saccharibacteria_unclassified_SGB19850', 's__Anaerotruncus_colihominis', 's__Prevotella_copri_clade_C', 's__Prevotella_stercorea', 's__Veillonella_rogosae', 's__Roseburia_sp_AM23_20', 's__Phocea_massiliensis', 's__GGB9593_SGB15015', 's__Mediterraneibacter_sp_gm002', 's__Blautia_hansenii', 's__GGB9512_SGB14909', 's__GGB1495_SGB2071', 's__Ruminococcaceae_bacterium_AM07_15', 's__GGB2982_SGB3964', 's__Bifidobacterium_dentium', 's__Holdemanella_biformis', 's__Bacilli_bacterium', 's__GGB3278_SGB4328', 's__Eisenbergiella_tayi', 's__Blautia_faecicola', 's__Sutterella_SGB9281', 's__Ruminococcaceae_unclassified_SGB15236', 's__Blautia_caecimuris', 's__Clostridiaceae_unclassified_SGB4769', 's__Negativibacillus_massiliensis', 's__GGB9634_SGB15093', 's__Desulfovibrionaceae_bacterium', 's__GGB3490_SGB4664', 's__GGB51441_SGB71759', 's__GGB4585_SGB6340', 's__Eubacterium_sp_AM28_29', 's__Enterocloster_clostridioformis', 's__Peptococcaceae_bacterium', 's__Enterocloster_asparagiformis', 's__Clostridium_SGB4751', 's__Christensenellaceae_bacterium', 's__Gordonibacter_pamelaeae', 's__Phocaeicola_coprophilus', 's__Parabacteroides_johnsonii', 's__Phascolarctobacterium_succinatutens', 's__Roseburia_sp_AM59_24XD', 's__Streptococcus_anginosus', 's__Sellimonas_intestinalis', 's__Turicibacter_sanguinis', 's__Mediterraneibacter_butyricigenes', 's__GGB9595_SGB15019']

def print_full_df(x):
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", None)
    pd.set_option("display.max_colwidth", None)
    display(x)
    pd.reset_option("display.max_rows")
    pd.reset_option("display.max_columns")
    pd.reset_option("display.width")
    pd.reset_option("display.float_format")
    pd.reset_option("display.max_colwidth")

# Pre-processing

### Preprocessing before splitting

In [ ]:
# Get sample group data
sample_group = pd.read_table(f"{data_root_dir}/sample.group", sep="\t", header=0)
# Remove studies
logger.info(f"sample_group.shape before removal of studies: {sample_group.shape}")
sample_group = sample_group[~sample_group["Project_1"].isin(studies_to_remove)]
logger.info(f"sample_group.shape after removal of studies: {sample_group.shape}")

# Keep recommended columns
logger.info(f"sample_group.shape before column removal: {sample_group.shape}")
sample_group = sample_group[columns_to_keep]
logger.info(f"sample_group_useful.shape after column removal: {sample_group.shape}")
# Set index to Sample
sample_group = sample_group.set_index("Sample")
logger.info(f"sample_group_useful.shape after setting index: {sample_group.shape}")

# Get species profile data
mpa4_profile = pd.read_table(
    f"{data_root_dir}/mpa4_genus.profile", sep="\t", header=0, index_col=0
)
# Remove species with no reads
mpa4_profile = mpa4_profile.loc[
    :, mpa4_profile.sum(axis=0) >= 1
]

## Remove repeated samples
logger.info(f"sample_group_useful.shape before removal: {sample_group.shape}")
sample_group = sample_group[~sample_group.index.duplicated(keep="first")]
logger.info(f"sample_group_useful.shape after removal: {sample_group.shape}")

# remove samples not in sample_group
logger.info(
    f"mpa4_species_profile.shape before filtering out samples without metadata: {mpa4_profile.shape}"
)
samples_to_keep = list(
    set(sample_group.index.tolist()) & set(mpa4_profile.columns.tolist())
)
mpa4_profile = mpa4_profile[samples_to_keep]
logger.info(
    f"mpa4_species_profile.shape after filtering out samples without metadata: {mpa4_profile.shape}"
)
mpa4_profile = mpa4_profile.T
logger.info(
    f"mpa4_species_profile.shape after transposing: {mpa4_profile.shape}"
)

# remove samples from sample_group that are not in mpa4_species_profile
logger.info(
    f"sample_group_useful.shape before filtering out samples not in mpa4_species_profile: {sample_group.shape}"
)
sample_group = sample_group.loc[samples_to_keep]
logger.info(
    f"sample_group_useful.shape after filtering out samples not in mpa4_species_profile: {sample_group.shape}"
)

# Normalize the data
logger.info(
    f"mpa4_species_profile summation before normalization: {mpa4_profile.sum(axis=1)}"
)
mpa4_profile = preprocessing_functions.total_sum_scaling(mpa4_profile)
logger.info(
    f"mpa4_species_profile summation after normalization: {mpa4_profile.sum(axis=1)}"
)


2025-02-18 14:54:56.210 | INFO     | __main__:<module>:4 - sample_group.shape before removal of studies: (6616, 21)
2025-02-18 14:54:56.219 | INFO     | __main__:<module>:6 - sample_group.shape after removal of studies: (6463, 21)
2025-02-18 14:54:56.221 | INFO     | __main__:<module>:9 - sample_group.shape before column removal: (6463, 21)
2025-02-18 14:54:56.223 | INFO     | __main__:<module>:11 - sample_group_useful.shape after column removal: (6463, 4)
2025-02-18 14:54:56.227 | INFO     | __main__:<module>:14 - sample_group_useful.shape after setting index: (6463, 3)
2025-02-18 14:54:59.270 | INFO     | __main__:<module>:26 - sample_group_useful.shape before removal: (6463, 3)
2025-02-18 14:54:59.272 | INFO     | __main__:<module>:28 - sample_group_useful.shape after removal: (6310, 3)
2025-02-18 14:54:59.275 | INFO     | __main__:<module>:31 - mpa4_species_profile.shape before filtering out samples without metadata: (1179, 8620)
2025-02-18 14:54:59.312 | INFO     | __main__:<modul

In [ ]:
# prevalence and abundance filtering
# low abundance filtering per study
grouped_sample_group = sample_group.groupby("Project_1")
abundance_filtering_threshold = 0.0001
display(mpa4_profile)
for project, samples in grouped_sample_group.groups.items():
    logger.info(f"Project: {project}")
    rows_to_update = mpa4_profile.loc[samples]
    # Skipping prevalence filtering
    # feature_prevalence = (rows_to_update > abundance_filtering_threshold).sum(axis=0) / rows_to_update.shape[0]
    # low_abundance_features = feature_prevalence < 0.1

    df_masked = rows_to_update.mask(
        # low_abundance_features | 
        (rows_to_update <= abundance_filtering_threshold), 0
    )
    mpa4_profile.update(df_masked)

display(mpa4_profile)
display(mpa4_profile.sum(axis=1))
display(mpa4_profile.sum(axis=1).sort_values(ascending=True))

# save it all
mpa4_profile.to_csv(
    f"{data_root_dir}/mpa4_genus_profile_after_abundane_prevalence_filtering.csv"
)

name,g__Phocaeicola,g__Faecalibacterium,g__Bacteroides,g__Ruminococcus,g__Clostridium,g__Lachnospiraceae_unclassified,g__Lachnospira,g__Roseburia,g__Clostridia_unclassified,g__Phascolarctobacterium,...,g__Sphingobium,g__Agrococcus,g__Candidatus_Sulfotelmatobacter,g__Aspergillus,g__Roseobacter,g__Thermus,g__Buttiauxella,g__Thermobifida,g__Rodentibacter,g__Desulfobulbus
SRR12000208,0.126040,0.104168,0.044533,0.003377,0.033819,0.074033,0.022016,0.044088,0.013315,0.014045,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LD-25,0.169447,0.092121,0.327071,0.054479,0.010358,0.034820,0.007015,0.027954,0.004289,0.008102,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR11992769,0.064642,0.082530,0.029992,0.004427,0.021684,0.010620,0.012481,0.042354,0.016721,0.027500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR13327549,0.004882,0.022360,0.003392,0.000614,0.017953,0.001474,0.000000,0.007582,0.000324,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR10403508,0.255034,0.132798,0.043195,0.022298,0.059294,0.007523,0.099071,0.133527,0.002949,0.017673,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR9038099,0.011448,0.078311,0.112935,0.004297,0.020476,0.042588,0.007385,0.002047,0.165319,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yu_5,0.019144,0.005669,0.042367,0.003856,0.003608,0.006766,0.051646,0.000526,0.129221,0.008284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR9075352,0.101948,0.090126,0.123341,0.009415,0.002299,0.016009,0.038844,0.083962,0.001573,0.007370,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR13077794,0.000000,0.000000,0.205626,0.000000,0.031214,0.025321,0.000000,0.016573,0.029801,0.017485,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


2025-02-18 14:55:40.186 | INFO     | __main__:<module>:6 - Project: ChenB_2020
2025-02-18 14:55:41.768 | INFO     | __main__:<module>:6 - Project: ChuY_2021
2025-02-18 14:55:43.870 | INFO     | __main__:<module>:6 - Project: HanL_2021
2025-02-18 14:55:45.862 | INFO     | __main__:<module>:6 - Project: HeQ_2017
2025-02-18 14:55:47.683 | INFO     | __main__:<module>:6 - Project: HuY_2019
2025-02-18 14:55:49.599 | INFO     | __main__:<module>:6 - Project: HuangR_2020
2025-02-18 14:55:51.418 | INFO     | __main__:<module>:6 - Project: JieZ_2017
2025-02-18 14:55:53.388 | INFO     | __main__:<module>:6 - Project: LiJ_2017
2025-02-18 14:55:55.122 | INFO     | __main__:<module>:6 - Project: LiR_2021
2025-02-18 14:55:56.981 | INFO     | __main__:<module>:6 - Project: LiuP_2021
2025-02-18 14:55:58.547 | INFO     | __main__:<module>:6 - Project: LiuR_2017
2025-02-18 14:56:00.022 | INFO     | __main__:<module>:6 - Project: LuW_2018
2025-02-18 14:56:01.501 | INFO     | __main__:<module>:6 - Project

name,g__Phocaeicola,g__Faecalibacterium,g__Bacteroides,g__Ruminococcus,g__Clostridium,g__Lachnospiraceae_unclassified,g__Lachnospira,g__Roseburia,g__Clostridia_unclassified,g__Phascolarctobacterium,...,g__Sphingobium,g__Agrococcus,g__Candidatus_Sulfotelmatobacter,g__Aspergillus,g__Roseobacter,g__Thermus,g__Buttiauxella,g__Thermobifida,g__Rodentibacter,g__Desulfobulbus
SRR12000208,0.126040,0.104168,0.044533,0.003377,0.033819,0.074033,0.022016,0.044088,0.013315,0.014045,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LD-25,0.169447,0.092121,0.327071,0.054479,0.010358,0.034820,0.007015,0.027954,0.004289,0.008102,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR11992769,0.064642,0.082530,0.029992,0.004427,0.021684,0.010620,0.012481,0.042354,0.016721,0.027500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR13327549,0.004882,0.022360,0.003392,0.000614,0.017953,0.001474,0.000000,0.007582,0.000324,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR10403508,0.255034,0.132798,0.043195,0.022298,0.059294,0.007523,0.099071,0.133527,0.002949,0.017673,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR9038099,0.011448,0.078311,0.112935,0.004297,0.020476,0.042588,0.007385,0.002047,0.165319,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yu_5,0.019144,0.005669,0.042367,0.003856,0.003608,0.006766,0.051646,0.000526,0.129221,0.008284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR9075352,0.101948,0.090126,0.123341,0.009415,0.002299,0.016009,0.038844,0.083962,0.001573,0.007370,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR13077794,0.000000,0.000000,0.205626,0.000000,0.031214,0.025321,0.000000,0.016573,0.029801,0.017485,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


SRR12000208    0.987458
LD-25          0.983398
SRR11992769    0.998567
SRR13327549    0.998785
SRR10403508    0.999278
                 ...   
SRR9038099     0.965188
Yu_5           0.971323
SRR9075352     0.949121
SRR13077794    0.995759
SRR6066175     0.711810
Length: 6303, dtype: float64

ERR1620261     0.066127
SRR13077661    0.092671
SRR16124320    0.235723
SRR13077864    0.240482
SRR10403516    0.246404
                 ...   
ERR1190555     0.999982
Yu_11          0.999985
ERR1620313     1.000000
SRR6504899     1.000000
SRR13077801    1.000000
Length: 6303, dtype: float64

In [ ]:
# normalize and transform
logger.info(
    f"mpa4_species_profile summation before normalization: {mpa4_profile.sum(axis=1)}"
)
mpa4_profile = preprocessing_functions.total_sum_scaling(mpa4_profile)
logger.info(
    f"mpa4_species_profile summation after normalization: {mpa4_profile.sum(axis=1)}"
)

# Centered arcsine transform
logger.info(
    f"mpa4_species_profile summation before centered arcsine transform: {mpa4_profile.sum(axis=1)}"
)
mpa4_profile = preprocessing_functions.centered_arcsine_transform(
    mpa4_profile
)
logger.info(
    f"mpa4_species_profile summation after centered arcsine transform: {mpa4_profile.sum(axis=1)}"
)

2025-02-18 14:57:46.255 | INFO     | __main__:<module>:2 - mpa4_species_profile summation before normalization: SRR12000208    0.987458
LD-25          0.983398
SRR11992769    0.998567
SRR13327549    0.998785
SRR10403508    0.999278
                 ...   
SRR9038099     0.965188
Yu_5           0.971323
SRR9075352     0.949121
SRR13077794    0.995759
SRR6066175     0.711810
Length: 6303, dtype: float64
2025-02-18 14:57:46.340 | INFO     | __main__:<module>:6 - mpa4_species_profile summation after normalization: SRR12000208    1.0
LD-25          1.0
SRR11992769    1.0
SRR13327549    1.0
SRR10403508    1.0
              ... 
SRR9038099     1.0
Yu_5           1.0
SRR9075352     1.0
SRR13077794    1.0
SRR6066175     1.0
Length: 6303, dtype: float64
2025-02-18 14:57:46.372 | INFO     | __main__:<module>:11 - mpa4_species_profile summation before centered arcsine transform: SRR12000208    1.0
LD-25          1.0
SRR11992769    1.0
SRR13327549    1.0
SRR10403508    1.0
              ... 
SRR903

In [6]:
# Save the data
mpa4_profile.to_csv(f"{data_root_dir}/mpa4_genus_profile_preprocessed.csv")
sample_group.to_csv(f"{data_root_dir}/sample_group_genus_preprocessed.csv")

---
# Testing dataloader

In [ ]:
%load_ext autoreload
%autoreload 2

from torch import manual_seed
from torch.utils.data import DataLoader

manual_seed(0)

from src.data.sun_et_al import BinaryFewShotBatchSampler, MicrobiomeDataset

test_study = ["ChenB_2020"]
val_study = ["ChuY_2021"]

train_df = mpa4_profile.loc[
    sample_group.loc[~sample_group["Project_1"].isin(test_study + val_study)].index
]
assert train_df.shape[0] == 5892

test_df = mpa4_profile.loc[
    sample_group.loc[sample_group["Project_1"].isin(test_study)].index
]
assert test_df.shape[0] == 231

val_df = mpa4_profile.loc[
    sample_group.loc[sample_group["Project_1"].isin(val_study)].index
]
assert val_df.shape[0] == 180

meta_data = sample_group[["Group", "Project_1"]].rename(
    columns={"Project_1": "project", "Group": "label"}
)

train = MicrobiomeDataset(train_df, meta_data.loc[train_df.index])
test = MicrobiomeDataset(test_df, meta_data.loc[test_df.index])
val = MicrobiomeDataset(val_df, meta_data.loc[val_df.index])

sampler = BinaryFewShotBatchSampler(train, 50, True, True)
train_loader = DataLoader(train, batch_sampler=sampler)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# samples, labels = next(iter(train_loader))
# print(samples.shape)
# print(labels.shape)
# print(samples)
# print(labels)


97
157
[np.int64(2862), np.int64(2924), np.int64(2938), np.int64(2893), np.int64(2761), np.int64(2820), np.int64(2953), np.int64(2772), np.int64(2968), np.int64(2816), np.int64(2877), np.int64(2935), np.int64(2768), np.int64(2850), np.int64(2773), np.int64(2769), np.int64(2866), np.int64(2872), np.int64(2837), np.int64(2770), np.int64(2867), np.int64(2843), np.int64(2815), np.int64(2864), np.int64(3009), np.int64(2915), np.int64(2808), np.int64(2925), np.int64(2887), np.int64(2765), np.int64(2838), np.int64(2950), np.int64(2817), np.int64(2895), np.int64(2776), np.int64(2759), np.int64(2771), np.int64(2780), np.int64(2818), np.int64(2868), np.int64(2806), np.int64(2763), np.int64(2918), np.int64(2898), np.int64(2910), np.int64(2891), np.int64(2847), np.int64(2827), np.int64(2873), np.int64(2849), np.int64(3011), np.int64(2783), np.int64(2845), np.int64(3004), np.int64(2781), np.int64(2870), np.int64(3006), np.int64(2931), np.int64(2963), np.int64(2941), np.int64(2932), np.int64(2946), 

: 

In [ ]:
from src.preprocessing.functions import pandas_label_encoder

m = pandas_label_encoder(meta_data)
m = m.sort_index().reset_index(drop=True).groupby("project")[["label"]]
g = m.get_group(0)
g2 = g.groupby("label").groups

import numpy as np

np.array(list(g2.values())[0])

l = [
    1,
    2,
    3,
]
l.extend(np.array(list(g2.values())[0]))
print(l)

train.__getitem__(l[5])

(tensor([ 0.4558,  0.1662, -0.0019,  ..., -0.0019, -0.0019, -0.0019]),
 tensor(1.))

# Meta-Learning

c:\Users\shaya\Documents\TU_projects\master_thesis\.thesis_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from sklearn.preprocessing import QuantileTransformer

sun_et_al_abundance = pd.read_csv(
    f"{data_root_dir}/mpa4_species_profile_after_abundane_prevalence_filtering.csv",
    index_col=0,
    header=0,
)
sun_et_al_metadata = pd.read_csv(
    f"{data_root_dir}/sample_group_preprocessed.csv",
    index_col=0,
    header=0,
)

# normalization/transformation testing different methods
## study-wise quantile transformation
grouped_samples = sun_et_al_metadata.groupby(["Project_1"])
for group, idx in grouped_samples.groups.items():
    rows_df = sun_et_al_abundance.loc[idx, :]
    print(f"qunatile transform for group: {group}")
    rows_df = pd.DataFrame(
        QuantileTransformer(n_quantiles=rows_df.shape[0]).fit_transform(rows_df),
        columns=rows_df.columns,
        index=rows_df.index,
    )
    sun_et_al_abundance.update(rows_df)

sun_et_al_abundance.to_csv(
    f"{data_root_dir}/mpa4_species_profile_quantile_transformed.csv"
)


KeyboardInterrupt: 

In [13]:
import src.main_meta_learning as mml

mml.main(
    "src.models.models",
    "model2",
    "mpa4_species_profile_preprocessed.csv",
    "sample_group_species_preprocessed.csv",
    "",
    "JieZ_2017",
    outer_lr_range=(2, 2),
    inner_lr_range=(1e-2, 1e-2),
    inner_rl_reduction_factor=1,
    n_epochs=10,
    train_k_shot=10,
    n_gradient_steps=2, # TODO check why more gradient steps and parallel tasks gives nan for loss
    n_parallel_tasks=5,
    n_components_reduction_factor=0,
    use_cached_pca=False,
    do_normalization_before_scaling=True,
    scale_factor_before_training=100,
    loss_fn="BCELog",
    algorithm="Reptile"
)

# from torch.utils.data import TensorDataset

# # N = 2**10
# # X = torch.randn(N, 2)
# # y = (X.sum(dim=1) > 0).long()  # Label is 1 if sum > 0, else 0.
# # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# # # Define a simple model: 2 input features, one hidden layer with 10 units, 2 outputs.
# # model = torch.nn.Sequential(
# #     torch.nn.Linear(2, 10),
# #     torch.nn.ReLU(),
# #     torch.nn.Linear(10, 2),
# # ).to(device)

# # # Define the loss function.
# # loss_fn = nn.CrossEntropyLoss()

# # # Outer (meta) optimizer: using SGD.
# # meta_lr = 1  # Outer learning rate.

# # # Inner-loop learning rate (α).
# # inner_lr = 0.5

# # # Number of inner-loop gradient steps per task.
# # n_gradient_steps = 5

# # ## testing
# # train_dataset = TensorDataset(X, y)
# # # Use a small batch size so that each batch is treated as a "task".
# # train_dataloader = DataLoader(train_dataset, batch_size=40, shuffle=True)
# # N = 2**10
# # X = torch.randn(N, 2)
# # y = (X.sum(dim=1) > 0).long()  # Label is 1 if sum > 0, else 0.
# # test_dataset = TensorDataset(X, y)
# # test_dataloader = DataLoader(test_dataset, batch_size=40, shuffle=True)
# # meta_optimizer = SGD(model.parameters(), lr=inner_lr)
# # MAML = maml.MAML(
# #     model=model,
# #     train_n_gradient_steps=n_gradient_steps,
# #     eval_n_gradient_steps=n_gradient_steps,
# #     loss_fn=loss_fn,
# #     device=device,
# #     meta_optimizer=meta_optimizer,
# #     outer_lr_range=(1, 0.00001),
# #     inner_lr_range=(0.5, 0.00001),
# #     k_shot=10,
# # )

# # MAML.fit(train_dataloader=train_dataloader, n_epochs=100, n_parallel_tasks=5, evaluate_train=True, val_dataloader=test_dataloader)



2025-02-24 15:49:44.412 | SUCCESS  | src.main_meta_learning:main:218 - wandb init done
2025-02-24 15:49:45.491 | INFO     | src.models.reptile_with_l2l:fit:134 - Starting training with 770 iterations and 5 parallel tasks.
2025-02-24 15:49:45.548 | INFO     | src.models.reptile_with_l2l:evaluate:111 - Evaluation after epoch 0: Loss = 0.86
2025-02-24 15:49:45.549 | INFO     | src.models.reptile_with_l2l:evaluate:112 - Accuracy = 0.56, F1 = 0.72, Precision = 0.56, Recall = 0.99, ROC-AUC = 0.60
2025-02-24 15:49:52.985 | INFO     | src.models.reptile_with_l2l:fit:178 - Epoch 1 complete at iteration 77/770 with 5 parallel tasks and 380 total tasks. Reinitializing DataLoader for next epoch.
2025-02-24 15:49:53.029 | INFO     | src.models.reptile_with_l2l:evaluate:111 - Evaluation after epoch 1: Loss = 0.83
2025-02-24 15:49:53.031 | INFO     | src.models.reptile_with_l2l:evaluate:112 - Accuracy = 0.57, F1 = 0.72, Precision = 0.56, Recall = 1.00, ROC-AUC = 0.52
2025-02-24 15:50:01.157 | INFO   